In [26]:
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from scipy.optimize import brute
from scipy.optimize import minimize
import numpy as np
import plotly.graph_objects as go
from IPython.display import clear_output

# dataframe from json file
df = pd.read_json('calibration_data (6).txt')
df["metal_resistance"] = df["heater1_resistance"]

In [31]:

from turtle import delay


best_error = -1

def SH_model(data,a,b,c):
    return (1/(a+b*np.log(data)+c*(np.log(data))**3))-273.15

def final_model (data,a,b,c,k1,k2,k3):
    chamber_resistance =  data[1]
    metal_resistance = data[0]


    return k1*SH_model(metal_resistance,a,b,c)+k2*SH_model(chamber_resistance,a,b,c)+k3

def error_function(data):
    #iterate through the dataframe
    final_error = 0
    k1,k2,k3 = data
    global best_a
    global best_b
    global best_c
    global best_k1
    global best_k2
    global best_k3
    global best_error
    
    for i in range(len(df)):
        error = final_model(df.iloc[i],best_a,best_b,best_c,k1,k2,k3) - df.iloc[i]['real_temp']
        final_error += np.sqrt(error**2)

    if final_error < best_error or best_error == -1:
        best_error = final_error
        best_k1 = k1
        best_k2 = k2
        best_k3 = k3
        clear_output(wait=True)
        print("error:" + str(final_error))
        print("k1:" + str(k1))
        print("k2:" + str(k2))
        print("k3:" + str(k3))


    return final_error

fittedParameters, pcov = curve_fit(SH_model, df['metal_resistance'], df['real_temp'],p0=[0.47e-3,3.9e-4,-11e-7])
best_a,best_b,best_c= fittedParameters

#optimize.brute error_function
res = minimize(error_function,(1,0.05,5),method='Nelder-Mead',options={'disp': True})

# wait for the optimization to finish




# # minimize error_function manually
# for k1 in np.arange(0,5,0.1):
#     for k2 in np.arange(0,5,0.1):
#         for k3 in np.arange(-20,20,0.1):
#             error_function([k1,k2,k3])

error:7.1757608305769836
k1:1.024675852587987
k2:0.10158686118862445
k3:-7.134868557410222
Optimization terminated successfully.
         Current function value: 7.175761
         Iterations: 271
         Function evaluations: 519
best error:7.1757608305769836
float k1 = 1.024675852587987;
float k2 = 0.10158686118862445;
float k3 = -7.134868557410222;
float a = 0.003008955562155505;
float b = 0.00010709399484035514;
float c = 3.040292279272696e-05;


In [32]:
# print the results
print("best error:" + str(best_error))

print("float k1 = " + str(best_k1) + ";")
print("float k2 = " + str(best_k2) + ";")
print("float k3 = " + str(best_k3) + ";")

print("float a = " + str(best_a) + ";")
print("float b = " + str(best_b) + ";")
print("float c = " + str(best_c) + ";")

best error:7.1757608305769836
float k1 = 1.024675852587987;
float k2 = 0.10158686118862445;
float k3 = -7.134868557410222;
float a = 0.003008955562155505;
float b = 0.00010709399484035514;
float c = 3.040292279272696e-05;


: 

In [30]:
# plot the data
fig = px.scatter(df, x="metal_resistance", y="real_temp", color="chamber_resistance")
#add an scatter plot with the SH model
xplot = np.linspace(0.3,6,100)
yplot = SH_model(xplot,best_a,best_b,best_c)
fig.add_trace(go.Scatter(x=xplot,y=yplot,mode='lines',name='SH model'))

#add a trace for the final model for each chamber resistance,metal resistance pair in the dataframe
yplot2 = [final_model(x,best_a,best_b,best_c,best_k1,best_k2,best_k3) for x in df.values]
xplot2 = df['metal_resistance']
fig.add_trace(go.Scatter(x=xplot2,y=yplot2,mode='markers',name='final model'))


fig.show()

In [69]:
# calculate error in the data against SH-model
error = 0
for i in range(len(df)):
    error += np.sqrt(
        (final_model(df.iloc[i],best_a,best_b,best_c,best_k1,best_k2,best_k3) -  df.iloc[i]['real_temp'])**2
        )
print(error)

error = 0
for i in range(len(df)):
    error += np.sqrt(
        (SH_model(df.iloc[i]['metal_resistance'],best_a,best_b,best_c) - df.iloc[i]['real_temp'])**2
        )
error

3.9830914821553094


5.168345940795646

In [176]:
final_model(df.iloc[2],best_a,best_b,best_c,best_k1,best_k2)

63.42949918471434

In [175]:
df.iloc[2]['real_temp']

67.8

In [189]:
display(a,b,c)

0.00281085445209924

0.00023488321331200981

8.25649163366219e-06